<a href="https://colab.research.google.com/github/patelpooja363/q_practice_assignment/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sentence_transformers import SentenceTransformer, InputExample,losses,util
from torch.utils.data import DataLoader
import pandas as pd
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import evaluation
import numpy as np


In [2]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.6 MB/s 
     |████████████████████████████████| 4.2 MB 15.2 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 86 kB 2.5 MB/s 
     |████████████████████████████████| 596 kB 32.4 MB/s 
     |████████████████████████████████| 6.6 MB 35.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=6cd2b99f2ed87123855763cca537f2980461914bd54d6df4050a9a1658a0f817
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
train_data="/content/drive/MyDrive/assignment_dataset/msr_paraphrase_train.txt"
test_data="/content/drive/MyDrive/assignment_dataset/msr_paraphrase_test.txt"

In [7]:
df = pd.read_csv(train_data, sep="\t ")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
df.head()

,Quality\t#1ID #2ID\t#1String #2String
0,1\t702876\t702977\tAmrozi accused his brother ...
1,0\t2108705\t2108831\tYucaipa owned Dominick 's...
2,1\t1330381\t1330521\tThey had published an adv...
3,"0\t3344667\t3344648\tAround 0335 GMT , Tab sha..."
4,"1\t1236820\t1236712\tThe stock rose $ 2.11 , o..."


In [9]:
df[['quality', 'id1', 'id2', 'string1', 'string2']] = df['Quality\t#1ID #2ID\t#1String #2String'].str.split('\t', expand=True)

In [10]:
train_dataset=df.drop(columns='Quality\t#1ID #2ID\t#1String #2String')

In [11]:
train_dataset.head()

,quality,id1,id2,string1,string2
0,1,702876,702977,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr..."
1,0,2108705,2108831,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ..."
3,0,3344667,3344648,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set..."
4,1,1236820,1236712,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...


In [12]:
train_dataset.isna().sum()

quality    0
id1        0
id2        0
string1    0
string2    0
dtype: int64

In [13]:
train_dataset.count()

quality    4076
id1        4076
id2        4076
string1    4076
string2    4076
dtype: int64

In [6]:
df_test = pd.read_csv(test_data, sep="\t ")

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [7]:
df_test.head()

,Quality\t#1ID #2ID\t#1String #2String
0,1\t1089874\t1089925\tPCCW 's chief operating o...
1,1\t3019446\t3019327\tThe world 's two largest ...
2,1\t1945605\t1945824\tAccording to the federal ...
3,0\t1430402\t1430329\tA tropical storm rapidly ...
4,0\t3354381\t3354396\tThe company didn 't detai...


In [8]:
df_test[['quality', 'id1', 'id2', 'string1', 'string2']] = df_test['Quality\t#1ID #2ID\t#1String #2String'].str.split('\t', expand=True)

In [9]:
test_dataset=df_test.drop(columns='Quality\t#1ID #2ID\t#1String #2String')

In [10]:
test_dataset.count(),test_dataset.head()

(quality    1725
 id1        1725
 id2        1725
 string1    1725
 string2    1725
 dtype: int64,   quality      id1      id2  \
 0       1  1089874  1089925   
 1       1  3019446  3019327   
 2       1  1945605  1945824   
 3       0  1430402  1430329   
 4       0  3354381  3354396   
 
                                              string1  \
 0  PCCW 's chief operating officer , Mike Butcher...   
 1  The world 's two largest automakers said their...   
 2  According to the federal Centers for Disease C...   
 3  A tropical storm rapidly developed in the Gulf...   
 4  The company didn 't detail the costs of the re...   
 
                                              string2  
 0  Current Chief Operating Officer Mike Butcher a...  
 1  Domestic sales at both GM and No. 2 Ford Motor...  
 2  The Centers for Disease Control and Prevention...  
 3  A tropical storm rapidly developed in the Gulf...  
 4  But company officials expect the costs of the ...  )

In [11]:
#train_sen1=np.array(train_dataset.string1.values.tolist())
#train_sen2=np.array(train_dataset.string2.values.tolist())
test_sen1=test_dataset.string1.values.tolist()
test_sen2=test_dataset.string2.values.tolist()

In [12]:
type(test_sen1)

list

In [14]:
#train_label=np.array(train_dataset.quality.values,dtype=np.float32)
test_label=np.array(test_dataset.quality.values,dtype=np.float32)

In [96]:
#train_torch_l

tensor([1., 0., 1.,  ..., 1., 1., 0.])

In [15]:
import torch
#train_torch_l = torch.from_numpy(train_label)
test_torch_l=torch.from_numpy(test_label)

In [97]:
train_examples=[]
for i in range(len(train_dataset)):
  train_examples.append(InputExample(texts=[train_dataset.string1[i],train_dataset.string2[i]], label=train_torch_l[i]))
  #df_merge=[train_dataset.string1[i],train_dataset.string2[i]]
  #df_final.append(df_merge)


In [81]:
train_loss = losses.CosineSimilarityLoss(model)

In [98]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [99]:
sentences1 = test_sen1[0:500]
sentences2 = test_sen2[0:500]
scores = test_torch_l[0:500]

In [42]:
train_dataset.quality.value_counts()

1    2753
0    1323
Name: quality, dtype: int64

In [43]:
PATH="/content/drive/MyDrive/assignment_dataset"

In [16]:
import transformers

In [100]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

In [101]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,warmup_steps=100,optimizer_class=transformers.optimization.AdamW, optimizer_params= {'lr': 1e-03},save_best_model = True,show_progress_bar=True,output_path=PATH,
          epochs=5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/255 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration:   0%|          | 0/255 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration:   0%|          | 0/255 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration:   0%|          | 0/255 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [104]:
torch.save(model, model_path)

In [103]:
model_path="/content/drive/MyDrive/assignment_dataset/best_model_similarity.pth"

In [108]:
embedding=model.encode(["Is this a duplicate?", "Is this a copy?"])

In [111]:
embedding

array([[ 0.1957391 ,  0.07613214, -0.82070637, ..., -0.24391328,
         0.9899373 , -0.23784983],
       [ 0.19573896,  0.07613184, -0.82070684, ..., -0.24391358,
         0.9899368 , -0.23784964]], dtype=float32)

In [129]:
sen1="Is this a duplicate?"
sen2="Is this a copy?"

In [130]:
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding[0],embedding[1])
print("Sentence 1:", sen1)
print("Sentence 2:", sen2)
print("Similarity score:", cosine_scores.item())

Sentence 1: Is this a duplicate?
Sentence 2: Is this a copy?
Similarity score: 0.9999991655349731


In [131]:
sen3="Is this a duplicate?"
sen4="Don't Go."

In [132]:
embedding1=model.encode([sen3, sen4])

In [133]:
# compute similarity scores of two embeddings
cosine_scores1 = util.pytorch_cos_sim(embedding1[0],embedding1[1])
print("Sentence 1:", sen3)
print("Sentence 2:", sen4)
print("Similarity score:", cosine_scores1.item())

Sentence 1: Is this a duplicate?
Sentence 2: Don't Go.
Similarity score: 0.9999991655349731


In [136]:
cosine_scores

tensor([[1.0000]])

In [20]:
model1 = torch.load('/content/drive/MyDrive/assignment_dataset/best_model_similarity.pth')
model1.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [141]:
model1.encode([sen3, sen4])

array([[ 0.1957391 ,  0.07613214, -0.82070637, ..., -0.24391328,
         0.9899373 , -0.23784983],
       [ 0.19573995,  0.07613135, -0.82070744, ..., -0.24391235,
         0.9899373 , -0.23785026]], dtype=float32)

In [37]:
sent1 = test_sen1[500:600]
sent2 = test_sen2[500:600]
scores1 = test_torch_l[500:600].numpy()

In [39]:
import math
test_embeddings=[]
similarity_score=[]
for i in range(len(sent1)):
  texts=[sent1[i],sent2[i]]
  test_embed=model1.encode(texts)
  cosine_scores = util.pytorch_cos_sim(test_embed[0],test_embed[1])
  similarity_score.append(math.ceil(cosine_scores.item()))
  test_embeddings.append(test_embed)
print(len(test_embeddings))

100


In [40]:
type(np.array(similarity_score)),type(scores1)

(numpy.ndarray, numpy.ndarray)

In [33]:
from sklearn.metrics import confusion_matrix,classification_report

In [43]:
cm=confusion_matrix(scores1,np.array(similarity_score))
cr=classification_report(scores1,np.array(similarity_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
print(cm,cr)

[[ 0 30]
 [ 0 70]]               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        30
         1.0       0.70      1.00      0.82        70

    accuracy                           0.70       100
   macro avg       0.35      0.50      0.41       100
weighted avg       0.49      0.70      0.58       100

